# 多输入通道

In [1]:
import torch
from torch import nn
import sys
import d2lzh_pytorch as d2l

In [2]:
def corr2d_multi_in(X,k):
    #沿着X和k的第0维（通道维）分别计算再相加
    res=d2l.corr2d(X[0,:,:],k[0,:,:])
    for i in range(1,X.shape[0]):
        res+=d2l.corr2d(X[i,:,:],k[i,:,:])
    return res

In [3]:
#我们可以构造图5.4中的输入数组X、核数组K来验证互相关运算的输出
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

# 多输出通道

In [4]:
def corr2d_multi_in_out(X,K):
    # 对K的第0维遍历，每次同输入X做互相关计算。所有结果使用stack函数合并在一起
    return torch.stack([corr2d_multi_in(X,k) for k in K])

In [5]:
K = torch.stack([K, K + 1, K + 2])
K.shape # torch.Size([3, 2, 2, 2])

torch.Size([3, 2, 2, 2])

In [6]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

# 卷积层

In [7]:
def corr2d_multi_in_out_1x1(X,k):
    c_i,h,w=X.shape
    c_o=k.shape[0]
    X=X.view(c_i,h*w)
    k=k.view(c_o,c_i)
    Y=torch.mm(k,X)
    return Y.view(c_o,h,w)

In [8]:
#经验证，做$1\times 1$卷积时，以上函数与之前实现的互相关运算函数corr2d_multi_in_out等价
X = torch.rand(3, 3, 3)
K = torch.rand(2, 3, 1, 1)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().item() < 1e-6

True